In [1]:
# duck db
import os
os.chdir("/home/kai/workspace/DeepDocs_Project/datalake/")

In [2]:
from client import DuckDBClient

In [5]:
conn = DuckDBClient(
    database_path="/mnt/AI_NAS/datalake/db/catalog.duckdb",
    read_only=False,
)

In [ ]:
conn.create_table_from_parquet(
    "test",
    "/mnt/AI_NAS/datalake/**/*.parquet",
    hive_partitioning=True,
    union_by_name=True
)

In [9]:
test = DuckDBClient(
    database_path="/mnt/AI_NAS/datalake/db/catalog.duckdb",
    read_only=False,
)

In [6]:
conn

In [1]:
from datasets import load_from_disk
dataset = load_from_disk('/home/kai/workspace/DeepDocs_Project/datalake/downloads/export_4999_items')

In [ ]:
result = conn.execute("""
    SELECT * FROM read_parquet('/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/variant=*/**/data.parquet', union_by_name=True)
""").fetchdf()

In [ ]:
from openai import OpenAI
import time

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:9064/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [ ]:
# print model

models = client.models.list()
for model in models.data:
    print(model.id)

In [ ]:
import base64

def base64_encode(
    image_path,
):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
    return base64.b64encode(image_bytes).decode("utf-8")
image_path = "/home/kai/workspace/DeepDocs_Project/datalake/managers/test4.jpg"
base64_image = base64_encode(
    image_path,
)
    
messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract all layout elements. Reading order does not matter.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        },
    ]

In [ ]:


start_time = time.time()
completion = client.chat.completions.create(
    model="adapter",
    messages=messages,
    temperature=0.0,
)
end_time = time.time()
elapsed_time = end_time - start_time
tokens = completion.usage.total_tokens
tokens_per_second = tokens / elapsed_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Tokens: {tokens}")
print(f"Tokens per second: {tokens_per_second:.2f}")

In [ ]:
result = completion.choices[0].message.content

In [ ]:
target

In [ ]:
from transformers import Qwen2VLImageProcessor

In [ ]:
processor = Qwen2VLImageProcessor.from_pretrained("/mnt/AI_NAS/AI_MODEL/checkpoints/Qwen/Qwen2.5-VL-3B-Instruct")

In [ ]:
import numpy as np
image = Image.open(image_path)
inputs = processor(
    images=[image],
    return_tensors="np",
)

In [ ]:
inputs['pixel_values']

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import json
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
target = json.loads(result)

for item in target:
    class_ = item.get("type", "unknown")
    bbox = item.get("bbox", [0, 0, 0, 0])
    x0, y0, x1, y1 = bbox
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    # 
    x0 -= 14
    x1 += 14
    y0 -= 14
    y1 += 14
    draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
    draw.text((x0, y0), class_, fill="red")    

In [ ]:
import sys
sys.path.append("/home/kai/workspace/DeepDocs_Project/datalake")
from client.src.core.duckdb_client import DuckDBClient
from datalake_client import DatalakeClient
from datasets import Dataset, load_from_disk
import os
import traceback
from datetime import datetime
from glob import glob
import pandas as pd
from tqdm import tqdm
# datalake_client.py
manager = DatalakeClient(
    nas_api_url="http://192.168.20.62:8091",
    num_proc=32)
duckdb = DuckDBClient(
    database_path=manager.base_path / "catalog.duckdb",
    read_only=True
)

In [ ]:
manager.show_nas_dashboard()

In [ ]:
all_db = duckdb.retrieve_partitions()
all_db

In [ ]:
index = 30
provider = all_db.iloc[index]['provider']
dataset = all_db.iloc[index]['dataset']
print(f"provider: {provider}, dataset: {dataset}")
db = duckdb.retrieve_with_existing_cols(
    providers=provider,
    datasets=dataset,
)

print("db:")

db = db.drop(columns=['provider', 'dataset', 'task', 'variant'], errors='ignore')
db.head()

In [ ]:
task = "kie"
variant = "base_kie"
meta = {
    "lang": "ko",
    "src": "real",
}
staging_dir, job_id = manager.upload_task_data(
    data_file=db,
    provider=provider,
    dataset=dataset,
    task=task,
    variant=variant,
    dataset_description="설명을 변경해주세요.",
    overwrite=False,
    meta=meta,
)

In [ ]:
manager.trigger_nas_processing()

In [ ]:
manager.wait_for_job_completion("job_20250617_060301_119")

In [ ]:
error_list = []
for parquet_path in tqdm(parquets):
    try:
        db = Dataset.from_parquet(parquet_path)
        if "image_path" in db.column_names:
            db = db.map(lambda x: {"image_path": apply_image_path(x)})
        # if db에 date가 없을경우
        if "date" not in db.column_names:
            db = db.map(lambda x: {"date": datetime.now().strftime("%Y-%m-%d")})
        
        provider, dataset, task, variant, *etc = parquet_path.replace("/mnt/AI_NAS/datalake/catalog/","").split("/")[:-1]
        provider = provider.replace("provider=", "")
        dataset = dataset.replace("dataset=", "")
        task = task.replace("task=", "")
        variant = variant.replace("variant=", "")
        meta = {}
        for et in etc:
            key, value = et.split("=")
            meta[key] = value
        if "table" in variant:
            meta["mod"] = "table"        

        print(f"provider: {provider}, dataset: {dataset}, task: {task}, variant: {variant}, meta: {meta}")
        temp_path = f"./temp/{provider}/{dataset}/{task}/{variant}"
        db.save_to_disk(temp_path)
        staging_dir, job_id = manager.upload_raw_data(
            data_file=temp_path,
            provider=provider,
            dataset=dataset,
            dataset_description="설명을 변경해주세요.",
            original_source="링크를 입력해주세요.",
            overwrite=False,
        )
        # manager.trigger_nas_processing()
        # manager.wait_for_job_completion(job_id, timeout=1280000)
        # uploaded_db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/catalog/provider={provider}/dataset={dataset}/task=raw/variant=*/data.parquet")
        # uploaded_db.save_to_disk(temp_path)
        # manager.upload_task_data(
        #     data_file=temp_path,
        #     provider=provider,
        #     dataset=dataset,
        #     task=task,
        #     variant=variant,
        #     dataset_description="설명을 변경해주세요.",
        #     original_source="링크를 입력해주세요.",
        #     **meta,  # unpack meta dictionary
        # )
    except Exception as e:
        print(f"Error processing {parquet_path}: {e}")
        print(traceback.format_exc())
        error_list.append({
            "parquet_path": parquet_path,
            "error": str(e)
        })

In [ ]:
db

parquets = glob("/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/**/*.parquet", recursive=True)
all_parquets = parquets.copy()
print(len(parquets))
parquets = [p for p in parquets if "synthtabnet" not in p]
parquets = [p for p in parquets if "pubtables" not in p]
parquets = [p for p in parquets if "pubtabnet" not in p]
parquets = [p for p in parquets if "fintabnet" not in p]
parquets = [p for p in parquets if "vis_qa" not in p]
parquets = [p for p in parquets if "table_image_text_pair" not in p]
parquets = [p for p in parquets if "diverse_ocr_word" not in p]
parquets = [p for p in parquets if "diverse_ocr_char" not in p]
parquets = [p for p in parquets if "tourism_food_menu_board" not in p]
print(len(parquets))

# 역으로 구하기

reverse_parquets = [p for p in all_parquets if p not in parquets]
reverse_parquets = [p for p in all_parquets if p not in parquets]
parquets = reverse_parquets.copy()
parquets = [p for p in parquets if "pubtabnet_otsl" not in p]
parquets = [p for p in parquets if "tourism_food_menu_board" not in p]
parquets = [p for p in parquets if "table_image_text_pair" not in p]
print(len(parquets))

In [ ]:

def apply_image_path(row):
    if row['image_path']:
        path = f"/mnt/AI_NAS/datalake/{row['image_path']}"
        # check path exists
        if os.path.exists(path):
            return path
        raise FileNotFoundError(f"Image path does not exist: {path}")
    return None
error_list = []
for parquet_path in tqdm(parquets):
    try:
        db = Dataset.from_parquet(parquet_path)
        if "image_path" in db.column_names:
            db = db.map(lambda x: {"image_path": apply_image_path(x)})
        # if db에 date가 없을경우
        if "date" not in db.column_names:
            db = db.map(lambda x: {"date": datetime.now().strftime("%Y-%m-%d")})
        
        provider, dataset, task, variant, *etc = parquet_path.replace("/mnt/AI_NAS/datalake/catalog/","").split("/")[:-1]
        provider = provider.replace("provider=", "")
        dataset = dataset.replace("dataset=", "")
        task = task.replace("task=", "")
        variant = variant.replace("variant=", "")
        meta = {}
        for et in etc:
            key, value = et.split("=")
            meta[key] = value
        if "table" in variant:
            meta["mod"] = "table"        

        print(f"provider: {provider}, dataset: {dataset}, task: {task}, variant: {variant}, meta: {meta}")
        temp_path = f"./temp/{provider}/{dataset}/{task}/{variant}"
        db.save_to_disk(temp_path)
        staging_dir, job_id = manager.upload_raw_data(
            data_file=temp_path,
            provider=provider,
            dataset=dataset,
            dataset_description="설명을 변경해주세요.",
            original_source="링크를 입력해주세요.",
            overwrite=False,
        )
        # manager.trigger_nas_processing()
        # manager.wait_for_job_completion(job_id, timeout=1280000)
        # uploaded_db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/catalog/provider={provider}/dataset={dataset}/task=raw/variant=*/data.parquet")
        # uploaded_db.save_to_disk(temp_path)
        # manager.upload_task_data(
        #     data_file=temp_path,
        #     provider=provider,
        #     dataset=dataset,
        #     task=task,
        #     variant=variant,
        #     dataset_description="설명을 변경해주세요.",
        #     original_source="링크를 입력해주세요.",
        #     **meta,  # unpack meta dictionary
        # )
    except Exception as e:
        print(f"Error processing {parquet_path}: {e}")
        print(traceback.format_exc())
        error_list.append({
            "parquet_path": parquet_path,
            "error": str(e)
        })

In [ ]:
staging_dir, job_id = manager.upload_raw_data(
    data_file=temp_path,
    provider=provider,
    dataset=dataset,
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
)

In [ ]:
manager.show_nas_dashboard()

In [ ]:
manager.trigger_nas_processing()

In [ ]:
manager.wait_for_job_completion("job_20250616_074217_948", timeout=1280000, polling_interval=60)

In [ ]:
import shutil
shutil.move("/data.parquet", "/mnt/AI_NAS/datalake/test.parquet")

In [ ]:
manager.show_nas_dashboard()

In [ ]:
manager.trigger_nas_processing()

In [ ]:
manager.upload_raw_data(
    data_file="arrow경로",
    provider="provider_name",
    dataset="dataset_name",
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
)

In [ ]:
manager.upload_task_data(
    data_file=parquet_path,
    provider=provider,
    dataset=dataset,
    task=task,
    variant=variant,
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
    **meta,  # unpack meta dictionary
)

In [ ]:
import duckdb

In [ ]:
client = duckdb.connect()

In [ ]:

result = client.execute("""
    SELECT * FROM read_parquet('/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/variant=*/data.parquet', union_by_name=True, filename=True, hive_partitioning=True)
""").fetchdf()

In [ ]:
d = Dataset.from_pandas(result)

In [ ]:
d[0]

In [ ]:
# path에 /mnt/AI_NASd/datalake/assets 추가
d = d.map(lambda x: {"path": f"/mnt/AI_NAS/datalake/assets/{x['path']}"})

In [ ]:
# d의 filename이 exsts한지 확인
def check_image_exists(row):
    if row['path']:
        path = row['path']
        return os.path.exists(path)
    return False

In [ ]:
d = d.map(lambda x: {"image_exists": check_image_exists(x)})